In [11]:
#pandas version '2.2.2'
import pandas as pd
#numpy version '1.26.4'
import numpy as np

In [4]:
try:
    #google colab only - put data in a folder "/content/drive/MyDrive/capstone_data/"
    from google.colab import drive
    colab = 1
    print("Running in Google Colab, load data to \"/content/drive/MyDrive/capstone_data/\" to import it.")
    drive.mount('/content/drive')
except:
    colab = 0
    print("Load data to \"/capstone_data/\" to import it.")

Load data to "/capstone_data/" to import it.


In [5]:
if colab == 1:
    df_pres = pd.read_csv("/content/drive/MyDrive/capstone_data/1976-2020-president.csv")
elif colab == 0:
    df_pres = pd.read_csv("capstone_data/1976-2020-president.csv")

In [6]:
df_pres

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT
1,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"FORD, GERALD",REPUBLICAN,False,504070,1182850,20210113,NaN,REPUBLICAN
2,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"MADDOX, LESTER",AMERICAN INDEPENDENT PARTY,False,9198,1182850,20210113,NaN,OTHER
3,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"BUBAR, BENJAMIN """"BEN""""",PROHIBITION,False,6669,1182850,20210113,NaN,OTHER
4,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"HALL, GUS",COMMUNIST PARTY USE,False,1954,1182850,20210113,NaN,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4282,2020,WYOMING,WY,56,83,68,US PRESIDENT,"JORGENSEN, JO",LIBERTARIAN,False,5768,278503,20210113,NaN,LIBERTARIAN
4283,2020,WYOMING,WY,56,83,68,US PRESIDENT,"PIERCE, BROCK",INDEPENDENT,False,2208,278503,20210113,NaN,OTHER
4284,2020,WYOMING,WY,56,83,68,US PRESIDENT,NaN,NaN,True,1739,278503,20210113,NaN,OTHER
4285,2020,WYOMING,WY,56,83,68,US PRESIDENT,OVERVOTES,NaN,False,279,278503,20210113,NaN,OTHER


In [7]:
df_pres = df_pres[['year', 'state_po', 'party_simplified', 'candidatevotes', 'totalvotes']]
df_pres

,year,state_po,party_simplified,candidatevotes,totalvotes
0,1976,AL,DEMOCRAT,659170,1182850
1,1976,AL,REPUBLICAN,504070,1182850
2,1976,AL,OTHER,9198,1182850
3,1976,AL,OTHER,6669,1182850
4,1976,AL,OTHER,1954,1182850
...,...,...,...,...,...
4282,2020,WY,LIBERTARIAN,5768,278503
4283,2020,WY,OTHER,2208,278503
4284,2020,WY,OTHER,1739,278503
4285,2020,WY,OTHER,279,278503


In [8]:
df_pres = df_pres[df_pres['year'] >= 2000]
df_pres = df_pres[df_pres['year'] <= 2019]

In [9]:
df_pres = df_pres[df_pres['party_simplified'] != "OTHER"]
df_pres = df_pres[df_pres['party_simplified'] != "LIBERTARIAN"]

In [10]:
df_pres_pivoted = df_pres.pivot_table(
    index=['year', 'state_po', 'totalvotes'],
    columns='party_simplified',
    values='candidatevotes',
    aggfunc='sum'
).reset_index()

df_pres_pivoted.columns.name = None
df_pres_pivoted = df_pres_pivoted.rename(columns={'DEMOCRAT': 'DEMOCRAT_votes', 'REPUBLICAN': 'REPUBLICAN_votes'})

display(df_pres_pivoted)

,year,state_po,totalvotes,DEMOCRAT_votes,REPUBLICAN_votes
0,2000,AK,285560,79004,167398
1,2000,AL,1666272,692611,941173
2,2000,AR,921781,422768,472940
3,2000,AZ,1532016,685341,781652
4,2000,CA,10965822,5861203,4567429
...,...,...,...,...,...
250,2016,VT,320467,178573,95369
251,2016,WA,3317019,1742718,1221747
252,2016,WI,2976150,1382536,1405284
253,2016,WV,713051,188794,489371


In [12]:
#mb moved import to top

# Create a complete list of years from 2000 to 2019
all_years = pd.DataFrame({'year': np.arange(2000, 2020)})

# Get all unique states from the original data
all_states = pd.DataFrame({'state_po': df_pres_pivoted['state_po'].unique()})

# Create a DataFrame with all combinations of years and states
all_combinations = all_years.merge(all_states, how='cross')

# Merge the complete combinations with the pivoted data
df_pres_filled = pd.merge(all_combinations, df_pres_pivoted, on=['year', 'state_po'], how='left')

# Sort by state and year before forward filling
df_pres_filled = df_pres_filled.sort_values(by=['state_po', 'year'])

# Forward fill the missing values
df_pres_filled = df_pres_filled.ffill()

# Display the filled DataFrame
display(df_pres_filled)

,year,state_po,totalvotes,DEMOCRAT_votes,REPUBLICAN_votes
0,2000,AK,285560.0,79004.0,167398.0
51,2001,AK,285560.0,79004.0,167398.0
102,2002,AK,285560.0,79004.0,167398.0
153,2003,AK,285560.0,79004.0,167398.0
204,2004,AK,312598.0,111025.0,190889.0
...,...,...,...,...,...
815,2015,WY,250701.0,69286.0,170962.0
866,2016,WY,258788.0,55973.0,174419.0
917,2017,WY,258788.0,55973.0,174419.0
968,2018,WY,258788.0,55973.0,174419.0


In [13]:
df_pres = df_pres_filled

In [14]:
df_pres = df_pres.rename({'year':'Year', 'state_po':'State', 'totalvotes':'Total_Pres_Votes', 'DEMOCRAT_votes':'Dem_Pres_Votes', 'REPUBLICAN_votes':'Rep_Pres_Votes'}, axis = 'columns')

In [15]:
df_pres = df_pres.sort_values(by=['Year', 'State'])

In [17]:
df_pres = df_pres[df_pres['State'] != "DC"]

In [18]:
if colab == 1:
    # Export to CSV (save to Google Drive)
    df_pres.to_csv('/content/drive/My Drive/capstone_data_cleaned/df_pres.csv', index=False)
elif colab == 0:
    df_pres.to_csv('capstone_data_cleaned/df_pres.csv', index=False)